In [1]:
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from sqlalchemy import create_engine
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
connection_string = "postgres:postgres@localhost:5432/Fifa"
engine = create_engine(f'postgresql://{connection_string}')

In [5]:
def scrap_selections(countries, years):
    selection_players = []

    for country in countries:
        for year in years:
            current_page = f'https://www.national-football-teams.com/country/{countries[country]}/{year}/{country}.html'
            table = pd.read_html(current_page)[0]
            table.columns = table.columns.droplevel()
            table['National Team'] = country
            table['year'] = year
            selection_players.append(table)

    selection_players_df = pd.concat(selection_players)
    selection_players_df = selection_players_df.reset_index().drop(selection_players_df.
                                                                   columns[selection_players_df
                                                                           .columns.str.contains('unnamed', case=False)],
                                                                    axis=1).drop(['index'], axis=1)
    selection_players_df.columns = ['name', 'date_of_birth', 'position', 'current_club', 'f_matches', 'f_substitutions',
                                   'f_goals', 'nf_matches', 'nf_substitutions', 'nf_goals', 'national_team', 'year']
    selection_players_df.index.name = 'id'
    return selection_players_df

In [6]:
def csv_to_postgres(path, table_name, delimiter):
    df = pd.read_csv(path, sep=delimiter)
    df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

In [7]:
countries = {'Brazil':28, 'Mexico': 121, 'France': 67}
years = np.arange(2017, 2020)

players_df = scrap_selections(countries, years)
players_df.to_csv("fifa_selection_players.csv")

In [8]:
csv_to_postgres("fifa_ranking-2019-06-14.csv", "Countries", ';')

In [9]:
csv_to_postgres("fifa_selection_players.csv", "Players", ',')

In [10]:
pd.options.display.max_rows = 9999
joined_rank = pd.read_sql_query('SELECT pl.name, pl.national_team, pl.year, ct.rank, ct.rank_date\
                    FROM "Players" AS pl, "Countries" AS ct\
                    WHERE pl.year = EXTRACT(YEAR FROM ct.rank_date)\
                    AND pl.national_team = ct.country_full;', con=engine)

In [11]:
joined_rank

,name,national_team,year,rank,rank_date
0,Alisson,Brazil,2017,2,2017-03-09
1,Alisson,Brazil,2017,2,2017-12-21
2,Alisson,Brazil,2017,2,2017-02-09
3,Alisson,Brazil,2017,1,2017-06-01
4,Alisson,Brazil,2017,2,2017-09-14
5,Alisson,Brazil,2017,2,2017-01-12
6,Alisson,Brazil,2017,2,2017-10-16
7,Alisson,Brazil,2017,2,2017-07-06
8,Alisson,Brazil,2017,1,2017-05-04
9,Alisson,Brazil,2017,2,2017-11-23
